In [1]:
import pandas as pd
import geopandas as gpd
import altair as alt

In [118]:
all_sheets = pd.read_excel('https://docs.google.com/spreadsheets/d/1yZv9w9zRKwrGTaR-YzmAqMefw4wMlaXocejdxZaTs6w/export?format=xlsx', sheet_name=None)

In [119]:
print(all_sheets.keys())
#df = all_sheets['Jan31_2pm']
#df.head()

odict_keys(['Jan31_7pm', 'Jan31_2pm', 'Jan30_930pm', 'Jan30_11am', 'Jan29_9pm', 'Jan29_230pm', 'Jan29_130pm', 'Jan28_11pm', 'Jan28_6pm', 'Jan28_1pm', 'Jan27_830pm', 'Jan27_7pm', 'Jan27_9am', 'Jan26_11pm', 'Jan26_11am', 'Jan25_10pm', 'Jan25_12pm', 'Jan25_12am', 'Jan24_12pm', 'Jan24_12am', 'Jan23_12pm', 'Jan22_12pm', 'Jan22_12am'])


In [120]:
selection_sheets = list(all_sheets.keys())#[::3]
#print(selection_sheets)

In [121]:
#selection_sheets = ['Jan31_7pm', 'Jan30_930pm', 'Jan29_230pm', 'Jan28_11pm', 'Jan27_7pm', 'Jan26_11pm', 'Jan25_10pm', 'Jan24_12pm', 'Jan23_12pm', 'Jan22_12pm']

In [125]:
df_all = pd.DataFrame()
for sheet in selection_sheets:
    df = all_sheets[sheet]
    df.rename(columns={'Last Update':'Date'}, inplace=True)
    df.rename(columns={'Date last updated':'Date'}, inplace=True)    
    df_all = pd.concat((df_all, df), axis=0, sort=False)

In [126]:
df_all.columns

Index(['Province/State', 'Country/Region', 'Date', 'Confirmed', 'Deaths',
       'Recovered', 'Suspected', 'Demised', 'Country'],
      dtype='object')

In [127]:
df_all.replace(
    to_replace=['Ningxia', 'Inner Mongolia', 'Xinjiang', 'Tibet'], 
    value=['Nei Mongol', 'Ningxia Hui','Xinjiang Uygur', 'Xizang'],
    inplace=True
)
#df_all.fillna(0, inplace=True)

In [128]:
df_all.head()
df_all.Date.unique()

array(['2020-01-31T19:00:00.000000000', '2020-01-31T14:00:00.000000000',
       '2020-01-31T17:00:00.000000000', '2020-01-30T21:30:00.000000000',
       '2020-01-30T11:00:00.000000000', '2020-01-29T21:00:00.000000000',
       '2020-01-29T14:30:00.000000000', '2020-01-29T13:30:00.000000000',
       '2020-01-28T23:00:00.000000000', '2020-01-28T18:00:00.000000000',
       '2020-01-28T13:00:00.000000000', '2020-01-27T20:30:00.000000000',
       '2020-01-27T19:00:00.000000000', '2020-01-27T09:00:00.000000000',
       '2020-01-26T23:00:00.000000000', '2020-01-26T11:00:00.000000000',
       '2020-01-25T22:00:00.000000000', '2020-01-25T12:00:00.000000000',
       '2020-01-25T00:00:00.000000000', '2020-01-24T12:00:00.000000000',
       '2020-01-24T16:00:00.000000000', '2020-01-24T00:00:00.000000000',
       '2020-01-23T12:00:00.000000000', '2020-01-22T12:00:00.000000000',
       '2020-01-21T00:00:00.000000000'], dtype='datetime64[ns]')

In [129]:
df_all.head()

,Province/State,Country/Region,Date,Confirmed,Deaths,Recovered,Suspected,Demised,Country
0,Hubei,Mainland China,2020-01-31 19:00:00,7153.0,249.0,169.0,NaN,NaN,NaN
1,Zhejiang,Mainland China,2020-01-31 19:00:00,537.0,NaN,14.0,NaN,NaN,NaN
2,Guangdong,Mainland China,2020-01-31 19:00:00,436.0,NaN,10.0,NaN,NaN,NaN
3,Henan,Mainland China,2020-01-31 19:00:00,352.0,2.0,3.0,NaN,NaN,NaN
4,Hunan,Mainland China,2020-01-31 19:00:00,332.0,NaN,3.0,NaN,NaN,NaN


In [130]:
facet_order = df_all.sort_values(by='Confirmed', ascending=False)['Province/State'].astype(str).unique().tolist()

In [131]:
provinces = alt.Chart(df_all, width=125, height=100).mark_line(point=True).encode(
    x=alt.X('yearmonthdatehours(Date):T', title=None),
    #x='Date:T',
    y=alt.Y('Confirmed:Q', scale=alt.Scale(type='log')),
    order='Date:T',
    facet=alt.Facet('Province/State:N', columns=4, sort=facet_order),
    tooltip=[alt.Tooltip('yearmonthdatehours(Date):T', title='Date'), 'Confirmed:Q','Province/State:N']
).resolve_scale(
    y='independent',
    x='independent'
)

In [132]:
provinces

alt.Chart(...)

In [14]:
import hkvportal

In [15]:
dp = hkvportal.Service('https://tsws.hkvservices.nl/mangrove.ws/', '236ebLIeCDdnKbfGe956IvZetIv1')

Dataservice is recognized. https://tsws.hkvservices.nl/mangrove.ws/entry.asmx?WSDL will be used as portal


In [16]:
#dp.create_database(database='2019-nCoV')

In [17]:
dp.update_entry(
    database='2019-nCoV',
    key='22Jan_31Jan_provinces_line',
    data=provinces.to_html(),
    description='small multiples provices'
)

entry available at:
https://tsws.hkvservices.nl/mangrove.ws/data.ashx?function=dataportal.db.getdata&parameters={database:'2019-nCoV',key:'22Jan_31Jan_provinces_line'}&contentType=SET_BY_USER


In [133]:
provinces_death = alt.Chart(df_all.replace(0, pd.np.nan), width=125, height=100).mark_line(point=True).encode(
    x=alt.X('yearmonthdatehours(Date):T', title=None),
    #x='Date:T',
    y=alt.Y('Deaths:Q', scale=alt.Scale(type='log')),
    order='Date:T',
    facet=alt.Facet('Province/State:N', columns=4, sort=facet_order),
    tooltip=[alt.Tooltip('yearmonthdatehours(Date):T', title='Date'), 'Deaths:Q','Province/State:N']
).resolve_scale(
    y='independent',
    x='independent'
)
provinces_death

alt.Chart(...)

In [39]:
dp.update_entry(
    database='2019-nCoV',
    key='22Jan_31Jan_provinces_line_deaths',
    data=provinces_death.to_html(),
    description='small multiples provices'
)

entry available at:
https://tsws.hkvservices.nl/mangrove.ws/data.ashx?function=dataportal.db.getdata&parameters={database:'2019-nCoV',key:'22Jan_31Jan_provinces_line_deaths'}&contentType=SET_BY_USER


In [134]:
provinces_recovered = alt.Chart(df_all.replace(0, pd.np.nan), width=125, height=100).mark_line(point=True).encode(
    x=alt.X('yearmonthdatehours(Date):T', title=None),
    #x='Date:T',
    y=alt.Y('Recovered:Q', scale=alt.Scale(type='log')),
    order='Date:T',
    facet=alt.Facet('Province/State:N', columns=4, sort=facet_order),
    tooltip=[alt.Tooltip('yearmonthdatehours(Date):T', title='Date'), 'Recovered:Q','Province/State:N']
).resolve_scale(
    y='independent',
    x='independent'
)
provinces_recovered

alt.Chart(...)

In [42]:
dp.new_entry(
    database='2019-nCoV',
    key='22Jan_31Jan_provinces_line_recovered',
    data=provinces_recovered.to_html(),
    description='small multiples provices'
)

entry available at:
https://tsws.hkvservices.nl/mangrove.ws/data.ashx?function=dataportal.db.getdata&parameters={database:'2019-nCoV',key:'22Jan_31Jan_provinces_line_recovered'}&contentType=SET_BY_USER


In [154]:
symbs_conf = alt.Chart(df_all.replace(pd.np.nan, None)).mark_circle(
    opacity=0.8,
    stroke='black',
    strokeWidth=1
).encode(
    alt.X('yearmonthdate(Date):T', axis=alt.Axis(labelAngle=0), title=None),
    alt.Y('Province/State:N', sort=facet_order),
    alt.Size(field='Confirmed', type='quantitative', aggregate='max',
        scale=alt.Scale(range=[10,4000],type='linear'),
        legend=alt.Legend(title='Confirmed Cases')
    ),
    alt.Color('Province/State:N', legend=None, scale=alt.Scale(scheme='tableau20')),
    tooltip=[alt.Tooltip('yearmonthdate(Date):T', title='Date'), alt.Tooltip('max(Confirmed):Q', title='Confirmed'),'Province/State:N']
).properties(
    width=250,
    height=450
)

In [155]:
symbs_conf

alt.Chart(...)

In [156]:
symbs_death = alt.Chart(df_all.replace(pd.np.nan, None)).mark_circle(
    opacity=0.8,
    stroke='black',
    strokeWidth=1
).encode(
    alt.X('yearmonthdate(Date):T', axis=alt.Axis(labelAngle=0), title=None),
    alt.Y('Province/State:N', sort=facet_order),
    alt.Size(field='Deaths', type='quantitative', aggregate='max',
        scale=alt.Scale(range=[10,2000],type='linear'),
        legend=alt.Legend(title='Casualties')
    ),
    alt.Color('Province/State:N', legend=None, scale=alt.Scale(scheme='tableau20')),
    tooltip=[alt.Tooltip('yearmonthdate(Date):T', title='Date'), alt.Tooltip('max(Deaths):Q', title='Deaths'),'Province/State:N']
).properties(
    width=250,
    height=450
)

In [157]:
symbs_death

alt.Chart(...)

In [158]:
symbs_recv = alt.Chart(df_all.replace(pd.np.nan, None)).mark_circle(
    opacity=0.8,
    stroke='black',
    strokeWidth=1
).encode(
    alt.X('yearmonthdate(Date):T', axis=alt.Axis(labelAngle=0), title=None),
    alt.Y('Province/State:N', sort=facet_order),
    alt.Size(field='Recovered', type='quantitative', aggregate='max',
        scale=alt.Scale(range=[10,2000],type='linear'),
        legend=alt.Legend(title='Recovered')
    ),
    alt.Color('Province/State:N', legend=None, scale=alt.Scale(scheme='tableau20')),
    tooltip=[alt.Tooltip('yearmonthdate(Date):T', title='Date'), alt.Tooltip('max(Recovered):Q', title='Deaths'),'Province/State:N']
).properties(
    width=250,
    height=450
)

In [161]:
symbs_all = (symbs_conf | symbs_death | symbs_recv).resolve_scale(size='independent')

In [162]:
dp.new_entry(
    database='2019-nCoV',
    key='22Jan_31Jan_provinces_symbol_all',
    data=symbs_all.to_html(),
    description='small multiples provices'
)

entry available at:
https://tsws.hkvservices.nl/mangrove.ws/data.ashx?function=dataportal.db.getdata&parameters={database:'2019-nCoV',key:'22Jan_31Jan_provinces_symbol_all'}&contentType=SET_BY_USER
